# Imports

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
import matplotlib.dates as mdates
import seaborn as sns
sns.set(context='talk',font='Courier New',font_scale=1.6)

In [ ]:
import plotly as py
import plotly.graph_objs as go

In [ ]:
import cufflinks as cf
cf.offline.go_offline(connected=True)

# Load and prepare the data

In [ ]:
data = pd.read_csv('../../data/measles_incidence.csv', skiprows=2, na_values='-')

In [ ]:
annual = data.drop('WEEK', axis=1).groupby('YEAR').sum()

In [ ]:
annual.to_csv('../../data/measles_incidence_per_year.csv')

In [ ]:
annual.head()

In [ ]:
stacked = annual.stack().reset_index()
stacked.columns = ['year', 'state', 'incidence']

In [ ]:
stacked.head()

# Average incidence over time

In [ ]:
# avg_incidence = stacked.groupby('year').incidence.mean()
avg_incidence = annual.mean(axis=1)

In [ ]:
trace = go.Scatter(x=avg_incidence.index,
                   y=avg_incidence,
                   mode='lines+markers')

layout = dict(title='Average Measles Incidence vs Time',
              xaxis=dict(title='Year'),
              yaxis=dict(title='Average Measles Incidence'),
              annotations=[dict(x=1963,
                                y=0.1,
                                xref='x',
                                yref='paper',
                                text='vaccine<br>introduced',
                                showarrow=True,
                                ax=0,
                                ay=-250)])

fig = go.Figure(data=[trace], layout=layout)

py.offline.iplot(fig)

In [ ]:
py.plotly.plot(fig, filename='measles_vs_time', sharing='public')

# Incidence over time for each U.S. state

In [ ]:
trace = go.Heatmap(z=annual.T.values,
                   x=annual.index,
                   y=annual.columns,
                   colorbar = dict(title = 'Incidence<br>(cases per 100K)',
                                   titleside = 'top',
                                   ticks = 'outside'))

layout = go.Layout(title='Annual Measles Incidence vs Time',
                   height=600,
                   xaxis=dict(title='Year',
                              tickmode='array',
                              tickvals=list(range(1920,2010,10))),
                   yaxis=dict(autotick=False,
                              tickfont=dict(size=8)),
                   margin=dict(l=120),
                   annotations=[dict(x=1963,
                                     y=1,
                                     xref='x',
                                     yref='paper',
                                     text='vaccine introduced',
                                     showarrow=True,
                                     ax=0)
                               ])

fig = go.Figure(data=[trace], layout=layout)

py.offline.iplot(fig)

In [ ]:
py.plotly.plot(fig, filename='measles_heatmap', sharing='public')

# Histogram of States by incidence

In [ ]:
annual.index

In [ ]:
year = 1942

trace = go.Histogram(x=annual.loc[year],
                     xbins=dict(start=0,
                                end=1500,
                                size=50),
                     name=str(year))

layout = go.Layout(title='U.S. States vs. Measles Incidence<br>year: '+str(year),
                   xaxis=dict(title='Measles Incidence (cases per 100K)',
                              range=[0,1500]),
                   yaxis=dict(title='Number of US states'),
                   bargap=0.05)

fig = go.Figure(data=[trace], layout=layout)

py.offline.iplot(fig)

In [ ]:
py.plotly.plot(fig, filename='measles_histogram_1942', sharing='public')

# Histogram of States by incidence for each year

In [ ]:
# Based on https://plot.ly/python/sliders/

# Traces and steps
traces = []
steps = []
for i, year in enumerate(annual.index):
    # Add a trace for every year
    trace = go.Histogram(x=annual.loc[year],
                         xbins=dict(start=0,
                                    end=1500,
                                    size=50),
                         name=str(year),
                         visible=True if year==1928 else False)
    traces.append(trace)
    
    # Add a step for every year
    step = dict(method='restyle',
                args=['visible', [False] * len(annual)],
                label=str(year))
    step['args'][1][i] = True # Toggle i'th trace to "visible"
    steps.append(step)

# Slider
sliders = [dict(active = 0,
                currentvalue = {"prefix": "Year: "},
                pad = {"t": 50},
                steps = steps)]

# Layout
layout = go.Layout(title='U.S. States vs. Measles Incidence',
                   xaxis=dict(title='Measles Incidence (cases per 100K)',
                              range=[0,1500]),
                   yaxis=dict(title='Number of US states',
                              range=[0,52]),
                   bargap=0.05,
                   sliders = sliders)

# Figure
fig = go.Figure(data=traces, layout=layout)

py.offline.iplot(fig)

In [ ]:
py.plotly.plot(fig, filename='measles_histograms', sharing='public')